In [56]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Concatenate, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.losses import Huber
from sklearn.preprocessing import StandardScaler
import random
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load your train_df
train_df = pd.read_csv('/content/drive/MyDrive/Modelling/train_df.csv')  # Change to your path


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image / 255.0  # Normalize to [0, 1]
    return image

# Prepare image data and scalar features
image_dir = '/content/drive/MyDrive/Modelling/valid_images'
image_paths = []
scalar_features = []
labels = []

for index, row in train_df.iterrows():
    pothole_id = row['pothole_id']
    image_path = os.path.join(image_dir, f'{pothole_id}.jpg')

    if os.path.exists(image_path):
        image_paths.append(image_path)
        scalar_features.append([row['pothole_area_mm2']])
        labels.append(row['bags_used'])

# Convert lists to numpy arrays
images = np.array([load_and_preprocess_image(path) for path in image_paths])
scalar_features = np.array(scalar_features)
labels = np.array(labels)

# Scale the scalar features
scaler = StandardScaler()
scalar_features = scaler.fit_transform(scalar_features)

# Save the scaler for later use
scaler_path = '/content/drive/MyDrive/Modelling/scaler.pkl'
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

# Embedding Layer

In [ ]:
def create_complex_cnn_model(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),

        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),  # Add dropout to reduce overfitting
        layers.Dense(256, activation='relu'),
    ])
    return model

def create_contrastive_model(input_shape):
    base_model = create_complex_cnn_model(input_shape)

    input_1 = layers.Input(shape=input_shape)
    input_2 = layers.Input(shape=input_shape)

    encoded_1 = base_model(input_1)
    encoded_2 = base_model(input_2)

    # Calculate the Euclidean distance between the two encodings
    distance = layers.Lambda(lambda tensors: tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=1, keepdims=True))([encoded_1, encoded_2])

    # Convert the distance to a similarity score
    similarity = layers.Dense(1, activation='sigmoid')(distance)

    model = models.Model(inputs=[input_1, input_2], outputs=similarity)
    return model

input_shape = (224, 224, 3)
contrastive_model = create_contrastive_model(input_shape)

# Compile the model
contrastive_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

images_train, images_val = train_test_split(images, test_size=0.2, random_state=42)

# Data Augmentation and Pairing
def augment_and_pair(image_1, image_2, label):
    # Augment both images in the pair
    augmented_1 = tf.image.random_flip_left_right(image_1)
    augmented_1 = tf.image.random_contrast(augmented_1, lower=0.8, upper=1.2)
    augmented_1 = tf.image.random_brightness(augmented_1, max_delta=0.2)

    augmented_2 = tf.image.random_flip_left_right(image_2)
    augmented_2 = tf.image.random_contrast(augmented_2, lower=0.8, upper=1.2)
    augmented_2 = tf.image.random_brightness(augmented_2, max_delta=0.2)

    return (augmented_1, augmented_2), label

# Create a dataset with both similar and dissimilar pairs
def create_pairs(images):
    pairs = []
    labels = []
    for i in range(len(images)):
        # Create a positive pair (similar)
        j = random.choice(range(len(images)))  # Pick a random index for a similar pair
        pairs.append((images[i], images[j]))
        labels.append(1)

        # Create a negative pair (dissimilar)
        k = random.choice(range(len(images)))  # Pick a random index for a dissimilar pair
        while k == i:
            k = random.choice(range(len(images)))
        pairs.append((images[i], images[k]))
        labels.append(0)

    return pairs, labels

def simclr_dataset(images, batch_size):
    pairs, labels = create_pairs(images)
    dataset = tf.data.Dataset.from_tensor_slices((pairs, labels))
    dataset = dataset.map(lambda x, y: augment_and_pair(x[0], x[1], y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

batch_size = 32
simclr_train_ds = simclr_dataset(images_train, batch_size)
simclr_val_ds = simclr_dataset(images_val, batch_size)

# Callbacks for early stopping and saving the best model
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Modelling/Checkpoints/best_contrastive_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Train the model
contrastive_model.fit(
    simclr_train_ds,
    epochs=50,
    validation_data=simclr_val_ds,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# Prediction Layer

In [48]:
base_model = contrastive_model.layers[2]  # Assuming the base model is the third layer
embeddings = base_model.predict(images_val)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step


In [50]:
!pip install xgb

ERROR: Could not find a version that satisfies the requirement xgb (from versions: none)
ERROR: No matching distribution found for xgb


In [52]:
def create_nn_regressor(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='linear')
    ])
    return model

# Get the input shape from the embeddings
input_shape = embeddings.shape[1:]

# Create the neural network regressor
nn_regressor = create_nn_regressor(input_shape)

# Compile the model
nn_regressor.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
nn_regressor.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping_callback])

# Predict on the test set
y_pred = nn_regressor.predict(X_test)

# Evaluate the model
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred))

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step - loss: 1.9179 - mae: 0.8688 - val_loss: 1.3321 - val_mae: 0.6001
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2201 - mae: 0.7420 - val_loss: 1.3347 - val_mae: 0.5814
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2196 - mae: 0.5777 - val_loss: 1.3409 - val_mae: 0.6198
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9556 - mae: 0.6658 - val_loss: 1.3314 - val_mae: 0.5975
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1795 - mae: 0.6086 - val_loss: 1.3313 - val_mae: 0.5966
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2946 - mae: 0.6025 - val_loss: 1.3327 - val_mae: 0.6020
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5487 - mae: 0.6403 - val_loss: 1.3316 - val_mae: 0.5893
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2189 - mae: 0.5893 - val_loss: 1.3360 - val_mae: 0.6084
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.82

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
-0.0005467976552211429


In [16]:
model.save('/content/drive/MyDrive/Modelling/saved_model/my_model.h5')